## Evaluation on FLEURS dataset

<a href="https://colab.research.google.com/github/shreyjasuja/re_s2st/blob/main/fleurs_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
from datasets import load_dataset
from tqdm import tqdm
import sacrebleu
import json

In [3]:
from pycountry import languages
def bcp47_to_iso639_3(bcp47_code):
    parts = bcp47_code.split('_')
    lang_code = parts[0]
    try:

      lang = languages.get(alpha_2=lang_code).alpha_3
      return lang.lower()
    except (AttributeError, KeyError) as e:
        # If the mapping fails, return the original code
        return lang_code.lower()

In [4]:
from datasets import get_dataset_config_names
bcp_47_codes=get_dataset_config_names("google/fleurs",trust_remote_code=True)

In [5]:
lang_dict={}
for i in bcp_47_codes:
  lang_dict[bcp47_to_iso639_3(i)]=i

'NoneType' object has no attribute 'alpha_3' ast
'NoneType' object has no attribute 'alpha_3' ceb
'NoneType' object has no attribute 'alpha_3' ckb
'NoneType' object has no attribute 'alpha_3' cmn
'NoneType' object has no attribute 'alpha_3' fil
'NoneType' object has no attribute 'alpha_3' kam
'NoneType' object has no attribute 'alpha_3' kea
'NoneType' object has no attribute 'alpha_3' luo
'NoneType' object has no attribute 'alpha_3' nso
'NoneType' object has no attribute 'alpha_3' umb
'NoneType' object has no attribute 'alpha_3' yue
'NoneType' object has no attribute 'alpha_3' all


In [7]:
!wget https://dl.fbaipublicfiles.com/seamless/metrics/evaluation_data_ids.zip -O evaluation_data_ids.zip

--2024-04-16 15:07:44--  https://dl.fbaipublicfiles.com/seamless/metrics/evaluation_data_ids.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.162.163.51, 3.162.163.34, 3.162.163.19, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.162.163.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6096377 (5.8M) [application/zip]
Saving to: ‘evaluation_data_ids.zip’

evaluation_data_ids 100%[===================>]   5.81M  --.-KB/s    in 0.08s   

2024-04-16 15:07:45 (73.7 MB/s) - ‘evaluation_data_ids.zip’ saved [6096377/6096377]



In [55]:

!unzip evaluation_data_ids.zip && rm evaluation_data_ids.zip

Archive:  evaluation_data_ids.zip
replace evaluation_data_ids/s2st_cvss_ids/test_cvss_jpn-eng.ids? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [8]:
old_codes=['msa','fil','uzb','fas','nep','lav','ara','aze','pus','ori','mon','swa','orm']
new_codes=['zlm','tgl','uzn','pes','npi','lvs','arb','azj','pbt','ory','khk','swh','gaz']
for i in range(len(new_codes)):
  lang_dict[new_codes[i]]=lang_dict[old_codes[i]]
  del lang_dict[old_codes[i]]

In [9]:
import os
base_path="evaluation_data_ids/s2tt_fleurs_ids/"
x_eng_files = [file for file in os.listdir("evaluation_data_ids/s2tt_fleurs_ids/") if file.endswith('-eng.ids')]
print(len(x_eng_files))

101


In [ ]:
from transformers import AutoProcessor, SeamlessM4TModel

processor = AutoProcessor.from_pretrained("facebook/hf-seamless-m4t-large")
model = SeamlessM4TModel.from_pretrained("facebook/hf-seamless-m4t-large")

model.cuda()

In [50]:
import collections
generated_translations=collections.defaultdict(dict)

In [11]:
src_lang_data=load_dataset("google/fleurs",name=lang_dict['ukr'],split="test",streaming=True)

/home/cc/.local/lib/python3.10/site-packages/datasets/load.py:1454: FutureWarning: The repository for google/fleurs contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/google/fleurs
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [12]:
print(next(iter(src_lang_data)))

{'id': 1982, 'num_samples': 118080, 'path': None, 'audio': {'path': 'test/10021730821550109934.wav', 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
       -1.80006027e-05, -2.40206718e-05, -9.59634781e-05]), 'sampling_rate': 16000}, 'transcription': 'жінки усім подорожнім жінкам радять казати що вони заміжні незалежно від справжнього сімейного стану', 'raw_transcription': 'Жінки: усім подорожнім жінкам радять казати, що вони заміжні, незалежно від справжнього сімейного стану.', 'gender': 0, 'lang_id': 92, 'language': 'Ukrainian', 'lang_group_id': 1}


In [26]:
lang_issue=[]
lang_missing_ids=[]

In [ ]:
import torch
import gc

for file_name in  x_eng_files:
  lang_code = file_name.split("-")[0].split("_")[-1]
  with open(base_path+file_name) as f:
    ids=f.read().split()

  if (lang_code in generated_translations.keys()) and (len(ids)==len(generated_translations[lang_code])) :
    print("Done")
    continue

  try:
    src_lang_data=load_dataset("google/fleurs",name=lang_dict[lang_code],split="test",streaming=True,trust_remote_code=True)
  except:
    lang_issue.append(lang_code)
    print("\n Missing language ",lang_code)
    continue



  for item in tqdm(src_lang_data,total=len(ids)):
    audio_sample = item['audio']
    id=audio_sample['path'].split("/")[-1].split(".")[0]

    if str(item['id'])+'_'+str(id) in generated_translations[lang_code]:
      continue

    if id not in ids:
      continue


    try:
        # Initially, try to process the audio on the GPU
        audio_inputs = processor(audios=audio_sample["array"], return_tensors="pt", sampling_rate=16000)
        audio_inputs = {k: v.to('cuda') for k, v in audio_inputs.items()}
        with torch.no_grad():
            output_tokens = model.generate(**audio_inputs, tgt_lang="eng", generate_speech=False)
        translation = processor.decode(output_tokens[0].tolist()[0], skip_special_tokens=True)
    except RuntimeError as e:
        if "CUDA out of memory" in str(e):
            print("\nCUDA out of memory. Shifting inference to CPU for ID:", id)
            torch.cuda.empty_cache()  # Clear any unreleased memory

            # Move the model to CPU for this inference
            model.to('cpu')

            try:
                # Make sure audio_inputs are on the CPU as well
                audio_inputs = {k: v.to('cpu') for k, v in audio_inputs.items()}
                with torch.no_grad():
                    output_tokens = model.generate(**audio_inputs, tgt_lang="eng", generate_speech=False)
                translation = processor.decode(output_tokens[0].tolist()[0], skip_special_tokens=True)
            except Exception as cpu_e:
                print("\nFailed processing on CPU for ID:", id, "with error:", cpu_e)
                lang_missing_ids.append((lang_code, id))
            finally:
                # Regardless of the outcome, put the model back on the GPU for subsequent operations
                model.to('cuda')
        else:
            print("\nAn error occurred for ID:", id, "Error:", e)
            lang_missing_ids.append((lang_code, id))
    except Exception as e:
        print("\nAn unexpected error occurred for ID:", id, "Error:", e)
        lang_missing_ids.append((lang_code, id))


    del audio_inputs
    del output_tokens

    generated_translations[lang_code][str(item['id'])+'_'+str(id)]=translation
  torch.cuda.empty_cache()

  with open ('generated_large.json','w')as f:
    json.dump(generated_translations,f,indent=2)






100%|██████████| 1015/1015 [12:48<00:00,  1.32it/s]
264it [03:08,  1.40it/s]
 63%|██████▎   | 237/379 [03:57<02:31,  1.07s/it]


CUDA out of memory. Shifting inference to CPU for ID: 3189556219205510204


 82%|████████▏ | 312/379 [05:56<00:58,  1.14it/s]


CUDA out of memory. Shifting inference to CPU for ID: 6634898757415929965


 14%|█▎        | 55/405 [00:42<04:34,  1.28it/s]


CUDA out of memory. Shifting inference to CPU for ID: 12560373056138365189


 66%|██████▌   | 554/842 [07:26<03:11,  1.51it/s]

In [ ]:
len(generated_translations)

101

In [22]:
def get_bleu_score(generated_translations):

  # get english data from fleurs dataset
  eng_data=load_dataset('google/fleurs',name='en_us',trust_remote_code=True)
  eng_translation={}
  for split in eng_data:
    for item in tqdm(eng_data[split]):
      audio_sample = item['audio']
      # id=audio_sample['path'].split("/")[-1].split(".")[0]
      eng_translation[item['id']]=item['raw_transcription']

  bleu_score={}
  for lang_code in list(generated_translations.keys()): #calculate bleu score for each language code

    translations=[]
    gt_translations=[]

    for i in generated_translations[lang_code]:
        key=int(i.split('_')[0])
        gt_translations.append(eng_translation[key])
        translations.append(generated_translations[lang_code][i])

    bleu = sacrebleu.corpus_bleu(translations, [gt_translations])
    bleu_score[lang_code]=round(bleu.score, 3)

  return bleu_score








In [11]:
import json
from collections import defaultdict

with open('generated_large.json') as f:
    generated_translations = json.load(f)

# Convert to defaultdict with empty dictionaries as default values
generated_translations = defaultdict(dict, generated_translations)

In [21]:
seamless_fleurs_bleu=get_bleu_score(generated_translations)

/home/cc/.local/lib/python3.10/site-packages/datasets/load.py:1454: FutureWarning: The repository for google/fleurs contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/google/fleurs
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
100%|██████████| 647/647 [00:00<00:00, 954.91it/s]


In [23]:
dict(sorted(seamless_fleurs_bleu.items()))

{'afr': 39.69,
 'amh': 17.034,
 'arb': 31.725,
 'asm': 17.47,
 'ast': 25.894,
 'azj': 16.425,
 'bel': 16.056,
 'ben': 22.778,
 'bos': 32.891,
 'bul': 31.33,
 'cat': 37.574,
 'ceb': 7.723,
 'ces': 31.016,
 'ckb': 20.487,
 'cmn': 18.98,
 'cym': 30.22,
 'dan': 33.553,
 'deu': 35.469,
 'ell': 24.804,
 'est': 28.534,
 'fin': 25.782,
 'fra': 32.641,
 'ful': 0.788,
 'gaz': 0.317,
 'gle': 10.654,
 'glg': 32.033,
 'guj': 27.164,
 'hau': 0.544,
 'heb': 28.226,
 'hin': 25.194,
 'hrv': 29.8,
 'hun': 24.166,
 'hye': 27.81,
 'ibo': 1.27,
 'ind': 28.81,
 'isl': 22.854,
 'ita': 25.307,
 'jav': 19.459,
 'jpn': 15.886,
 'kam': 1.803,
 'kan': 21.799,
 'kat': 18.741,
 'kaz': 21.338,
 'kea': 27.313,
 'khk': 16.258,
 'khm': 18.62,
 'kir': 16.771,
 'kor': 18.402,
 'lao': 19.088,
 'lin': 0.917,
 'lit': 20.675,
 'ltz': 14.429,
 'lug': 16.179,
 'luo': 0.789,
 'lvs': 27.666,
 'mal': 20.99,
 'mar': 21.372,
 'mkd': 33.972,
 'mlt': 38.23,
 'mri': 0.99,
 'mya': 14.676,
 'nld': 26.502,
 'nob': 33.007,
 'npi': 23.518,

In [24]:
with open('seamless_large_fleurs_bleu.json','w')as f:
  json.dump(seamless_fleurs_bleu,f)

In [13]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2")
model.to('cuda')
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
allowed_whisper_lang=['en', 'zh', 'de', 'es', 'ru', 'ko', 'fr', 'ja', 'pt', 'tr', 'pl', 'ca', 'nl', 'ar', 'sv', 'it', 'id', 'hi', 'fi', 'vi', 'he', 'uk', 'el', 'ms', 'cs', 'ro', 'da', 'hu', 'ta', 'no', 'th', 'ur', 'hr', 'bg', 'lt', 'la', 'mi', 'ml', 'cy', 'sk', 'te', 'fa', 'lv', 'bn', 'sr', 'az', 'sl', 'kn', 'et', 'mk', 'br', 'eu', 'is', 'hy', 'ne', 'mn', 'bs', 'kk', 'sq', 'sw', 'gl', 'mr', 'pa', 'si', 'km', 'sn', 'yo', 'so', 'af', 'oc', 'ka', 'be', 'tg', 'sd', 'gu', 'am', 'yi', 'lo', 'uz', 'fo', 'ht', 'ps', 'tk', 'nn', 'mt', 'sa', 'lb', 'my', 'bo', 'tl', 'mg', 'as', 'tt', 'haw', 'ln', 'ha', 'ba', 'jw', 'su', 'yue', 'my', 'ca', 'nl', 'ht', 'lb', 'ps', 'pa', 'ro', 'ro', 'si', 'es', 'zh']

In [18]:
import pycountry

def iso639_3_to_iso639_1(code_639_3):
    # Cross-mapping dictionary from provided lists
    cross_mapping = dict(zip(new_codes, old_codes))

    # Additional special cases for direct conversion from ISO 639-3 to ISO 639-1
    special_cases = {
        'cmn': 'zh',  # Mandarin Chinese
        'nob': 'no',  # Norwegian Bokmål
        'jav': 'jw'   # Javanese
        # Add any other special cases if needed
    }

    # Check special cases first
    if code_639_3 in special_cases:
        return special_cases[code_639_3]

    # Use cross-mapping to find the corresponding old code if available
    if code_639_3 in cross_mapping:
        code_639_3 = cross_mapping[code_639_3]

    # Use pycountry to attempt to convert any code to ISO 639-1 two-letter code
    try:
        language = pycountry.languages.get(alpha_3=code_639_3)
        return language.alpha_2 if hasattr(language, 'alpha_2') else code_639_3
    except AttributeError:
        # Return the original code if no ISO 639-1 code is found
        return code_639_3

In [19]:
whisper_codes={}
for code in lang_dict:
  x=iso639_3_to_iso639_1(code)
  if x in allowed_whisper_lang:
    whisper_codes[code]=x



In [52]:
len(whisper_codes)

82

In [45]:
import collections
generated_transcriptions=collections.defaultdict(dict)

In [29]:
options = dict(beam_size=5, best_of=5)

In [51]:
lang_issue=[]
lang_missing_ids=[]

In [ ]:
import torch
import gc

for file_name in  x_eng_files:
  lang_code = file_name.split("-")[0].split("_")[-1]
  with open(base_path+file_name) as f:
    ids=f.read().split()

  if (lang_code in generated_translations.keys()) and (len(ids)==len(generated_translations[lang_code])) :
    print("Done")
    continue



  try:

    forced_decoder_ids={
    'translate' : processor.get_decoder_prompt_ids(language=whisper_codes[lang_code], task="translate"),
    'transcribe': processor.get_decoder_prompt_ids(language=whisper_codes[lang_code], task="transcribe")
    }
    src_lang_data=load_dataset("google/fleurs",name=lang_dict[lang_code],split="test",streaming=True,trust_remote_code=True)

  except:
    lang_issue.append(lang_code)
    print("\n Missing language in whisper",lang_code)
    continue




  for item in tqdm(src_lang_data,total=len(ids)):
    audio_sample = item['audio']
    id=audio_sample['path'].split("/")[-1].split(".")[0]

    if str(item['id'])+'_'+str(id) in generated_translations[lang_code]:
      continue

    if id not in ids:
      continue

    try:
        # Initially, try to process the audio on the GPU
        input_features = processor(audio_sample["array"], sampling_rate=16000, return_tensors="pt").input_features
        input_features= input_features.to('cuda')
        with torch.no_grad():
          translate_output = model.generate(input_features, forced_decoder_ids=forced_decoder_ids['translate'])
          transcript_output = model.generate(input_features, forced_decoder_ids=forced_decoder_ids['transcribe'])

        translation = processor.batch_decode(translate_output, skip_special_tokens=True)
        transcription = processor.batch_decode(transcript_output, skip_special_tokens=True)
    except RuntimeError as e:
        if "CUDA out of memory" in str(e):
            print("\nCUDA out of memory. Shifting inference to CPU for ID:", id)
            torch.cuda.empty_cache()  # Clear any unreleased memory

            # Move the model to CPU for this inference
            model.to('cpu')

            try:
                input_features= input_features.to('cpu')
                with torch.no_grad():
                  translate_output = model.generate(input_features, forced_decoder_ids=forced_decoder_ids['translate'])
                  transcript_output = model.generate(input_features, forced_decoder_ids=forced_decoder_ids['transcribe'])

                translation = processor.batch_decode(predicted_ids, skip_special_tokens=True)
                transcription = processor.batch_decode(transcript_output, skip_special_tokens=True)
            except Exception as cpu_e:
                print("\nFailed processing on CPU for ID:", id, "with error:", cpu_e)
                lang_missing_ids.append((lang_code, id))
            finally:
                # Regardless of the outcome, put the model back on the GPU for subsequent operations
                model.to('cuda')
        else:
            print("\nAn error occurred for ID:", id, "Error:", e)
            lang_missing_ids.append((lang_code, id))
    except Exception as e:
        print("\nAn unexpected error occurred for ID:", id, "Error:", e)
        lang_missing_ids.append((lang_code, id))

    del input_features ,transcript_output, translate_output

    generated_translations[lang_code][str(item['id'])+'_'+str(id)]=translation
    generated_transcriptions[lang_code][str(item['id'])+'_'+str(id)]=transcription
  torch.cuda.empty_cache()

  with open ('generated_translations_whisper.json','w')as f:
    json.dump(generated_translations,f,indent=2)

  with open ('generated_transcriptions_whisper.json','w')as f:
    json.dump(generated_transcriptions,f,indent=2)






Done


100%|██████████| 775/775 [23:39<00:00,  1.83s/it]



 Missing language in whisper ckb


100%|██████████| 905/905 [30:33<00:00,  2.03s/it]



 Missing language in whisper kir


100%|██████████| 726/726 [41:45<00:00,  3.45s/it]



 Missing language in whisper wol


100%|██████████| 1015/1015 [58:49<00:00,  3.48s/it]



 Missing language in whisper ful


  8%|▊         | 67/851 [02:24<30:03,  2.30s/it]